# Amygdala single cell templates
## This notebook contains the major cells types present in the amygdala along with code to generate the passive properties, current injections, and FI curves for each cell. Also present in this notebook is the bio data in which these template cells were constrained from. 

Download the project directory

In [1]:
!rm -rf ./Amygdala-Cells
!git clone https://github.com/GregGlickert/Amygdala-Cells &> /dev/null

Install the necessary packages

In [2]:
!pip install --upgrade bmtool neuron &> /dev/null

Compile the mechanisms for the cell templates

In [3]:
!cd ./Amygdala-Cells/mechanisms && nrnivmodl modfiles &> /dev/null
%cd Amygdala-Cells/

/content/Amygdala-Cells


This section of code is run through a subprocess. Once neuron loads a hoc template file there is no reloading. By creating a subprocess Neuron is isolated and closes after running.

This allows for conductance changes to the `Amygdala/feng.hoc` and `Amygdala/som.hoc` files.


In [4]:
#@title Cell template gui code
from neuron import clear_gui_callback
from multiprocessing import Process
import ipywidgets as widgets
from ipywidgets import interactive_output,HBox,VBox,Label,Layout
from IPython.display import display
from IPython.display import clear_output

# Function to be run in a subprocess
def run(Cell = None, option = None, noise = None, inj_amp = 300, inj_delay = 400, inj_dur = 1000):
    # Build a profiler object
    from bmtool.singlecell import Profiler

    profiler = Profiler(template_dir='templates', 
                        mechanism_dir='mechanisms')
    if option == 'Passive properties':
      # Get passive properties for a given cell template you can change when passive properties are taken   
      _,_ = profiler.passive_properties(Cell, plot=False)
      print("BIO DATA")
      if Cell == 'Cell_Af' or Cell == 'Cell_Cf':
        print("RMP of -65.8 plus or minus 2.9 mV \ninput resistance of 58.1 plus or minus 10 megaohms \ntau of 27.9 plus or minus 1.9 ms")

    if option == 'Current Clamp':
      # Voltage trace with current clamp for a given cell template and injection strength
      if noise == True:
        time_vec, voltage_vec = profiler.current_injection(Cell, post_init_function="insert_mechs(123)", inj_amp=inj_amp, inj_delay=inj_delay, inj_dur=inj_dur,tstop=1700)
      if noise == False:
        time_vec, voltage_vec = profiler.current_injection(Cell, inj_amp=inj_amp, inj_delay=inj_delay, inj_dur=inj_dur,tstop=1700)

    if option == 'FI curve':
      # FI curve given a cell
      if noise == True:
        amp_vec, spike_vec = profiler.fi_curve(Cell, post_init_function="insert_mechs(123)")
      if noise == False:
        amp_vec, spike_vec = profiler.fi_curve(Cell)
    
    if option == 'All of the above':
      _,_ = profiler.passive_properties(Cell)
      if noise == True:
        time_vec, voltage_vec = profiler.current_injection(Cell, post_init_function="insert_mechs(123)", inj_amp=inj_amp, inj_delay=inj_delay, inj_dur=inj_dur,tstop=1700)
        amp_vec, spike_vec = profiler.fi_curve(Cell, post_init_function="insert_mechs(123)")
      if noise == False:
        time_vec, voltage_vec = profiler.current_injection(Cell, inj_amp=inj_amp, inj_delay=inj_delay, inj_dur=inj_dur,tstop=1700)
        amp_vec, spike_vec = profiler.fi_curve(Cell)

# runs the profiler without using the current clamp settings
def run_profiler(btn):
  clear_output()
  Cell = cell_select.value
  plot = plot_select.value
  Noise = noise.value
  p = Process(target=run(Cell=Cell, option=plot, noise = Noise))
  p.start()
  p.join()
  display(ui)

# runs the profiler using current clamp setting get separte to have everything look nicer
def run_profiler_clamp(btn):
  clear_output()
  Cell = cell_select.value
  plot = plot_select.value
  Noise = noise.value
  inj_amp = current_amp.value
  inj_dur = current_dur.value
  inj_delay = current_delay.value
  p = Process(target=run(Cell=Cell, option=plot, noise = Noise, inj_amp = inj_amp, inj_delay = inj_delay, inj_dur = inj_dur))
  p.start()
  p.join()
  display(ui)  

def current_settings(btn):
  run_clamp = widgets.Button(description="run clamp!",icon='history',button_style='primary')
  current_ui = HBox([current_amp, current_dur, current_delay, run_clamp])
  run_clamp.on_click(run_profiler_clamp)
  display(current_ui)

# core widgets
cell_select = widgets.RadioButtons(options=['Cell_Af', 'Cell_Cf', 'InterneuronCellf', 'SOM_Cell'],value='Cell_Af',description='Cell:',disabled=False)
plot_select = widgets.RadioButtons(options=['Passive properties', 'Current Clamp', 'FI curve', 'All of the above'],value='Passive properties',description='Plot:',disabled=False)
show = widgets.Button(description="Show result",icon='history',button_style='primary')
current_clamp_adv = widgets.Button(description="Clamp Settings",icon='history',button_style='primary')
noise = widgets.Checkbox(value=False,description='Check to add noise to cell',disabled=False,indent=False)

#current clamp widgets
current_amp = widgets.BoundedFloatText(value=300,min=-200,max=500.0,step=50,description='Current (pA)',disabled=False)
current_dur = widgets.BoundedFloatText(value=1000,min=0,max=1500,step=100,description='Duration',disabled=False)
current_delay = widgets.BoundedFloatText(value=400,min=0,max=1000,step=100,description='Delay',disabled=False)


show.on_click(run_profiler)
current_clamp_adv.on_click(current_settings)
ui = HBox([cell_select, plot_select, noise, VBox([show,current_clamp_adv])])
ui



The bio data for the Type A, Type C and PV(interneuronCellF) cells are from

Intracellular recordings from morphologically identified neurons of the basolateral amygdala (Rainnie et al., 1993)
https://journals.physiology.org/doi/epdf/10.1152/jn.1993.69.4.1350



with additional bio data for PV cells from Networks of Parvalbumin-Positive Interneurons in the Basolateral Amygdala (Woodruff and Sah, 2007)
https://www.jneurosci.org/content/27/3/553

PN cells

RMP of -65.8 plus or minus 2.9 mV

input resistance of 58.1 plus or minus 10 megaohms 

tau of 27.9 plus or minus 1.9 ms